In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

`gensim` provides a well-documented implementation of `word2vec`.

In [4]:
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec

Logistic regression is a sufficient classification method for word-vectors.

## Preprocessing
We first build a vocabulary table, which consists of

- filtering out unique and stopwords
- count-vectorizing

In [16]:
sentences = []

In [17]:
model = Doc2Vec(min_count=1, window=10, size=100, sample=1e-4, negative=5, workers=7)
model.build_vocab(sentences.to_array())

/usr/local/lib/python3.5/dist-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


AttributeError: 'list' object has no attribute 'to_array'

## Training
We train our model on an Amazon reviews dataset with <=3.6M reviews. `doc2vec` is really just an  ensemble of `word2vec` vectors, which themselves are numeric representations of individual words.